# Scratchwork to clean the data files so they can be imported to postgresql

In [2]:
folder_path = "sample_data"

files = [
    f for f in os.listdir(folder_path)
    if os.path.isfile(os.path.join(folder_path, f))
]


for filename in files:
    df = pd.read_csv('sample_data/'+filename)
    df["amount"] = pd.to_numeric(df["amount"], errors="coerce")
    df = df.dropna(subset=["merchant_id"])
    df.drop(df[df["event_timestamp"] == "NOT-A-DATE"].index, inplace=True)
    df.to_csv("clean_data/"+filename, index=False)

['activities_20240101.csv', 'activities_20240102.csv', 'activities_20240103.csv', 'activities_20240104.csv', 'activities_20240105.csv', 'activities_20240106.csv', 'activities_20240107.csv', 'activities_20240108.csv', 'activities_20240109.csv', 'activities_20240110.csv', 'activities_20240111.csv', 'activities_20240112.csv', 'activities_20240113.csv', 'activities_20240114.csv', 'activities_20240115.csv', 'activities_20240116.csv', 'activities_20240117.csv', 'activities_20240118.csv', 'activities_20240119.csv', 'activities_20240120.csv', 'activities_20240121.csv', 'activities_20240122.csv', 'activities_20240123.csv', 'activities_20240124.csv', 'activities_20240125.csv', 'activities_20240126.csv', 'activities_20240127.csv', 'activities_20240128.csv', 'activities_20240129.csv', 'activities_20240130.csv', 'activities_20240131.csv']


# Scratchwork to connect the database to python

In [22]:
from dotenv import load_dotenv
import os

load_dotenv()  # automatically loads .env from current directory

database = os.getenv("DATABASE")
user = os.getenv("USER")
password= os.getenv("PASSWORD")
table_name = os.getenv("TABLE_NAME")


In [23]:
import psycopg2

# Connect to PostgreSQL
conn = psycopg2.connect(
    host="localhost",
    database=database,
    user=user,
    password=password
)
cursor = conn.cursor()

In [18]:
# Execute a query
cursor.execute(f"SELECT * FROM {table_name}", (30,))

# Fetch results
rows = cursor.fetchmany(100) # changed this from fetchall(100) 

for row in rows:
    print(row)

('fc5a90a4-26c8-497a-9daa-dbd9af0fc2a9', 'MRC-000001', None, 'POS', 'CARD_TRANSACTION', Decimal('1000.0'), 'SUCCESS', 'POS', 'LAGOS', 'VERIFIED')
('fab56e86-398c-4e40-8709-2fb3e1a54a04', 'MRC-000001', datetime.datetime(2024, 1, 1, 10, 0), 'POS', 'CARD_TRANSACTION', None, 'SUCCESS', 'POS', 'LAGOS', 'VERIFIED')
('fbd09480-c00b-44cf-920d-6625d2550df8', 'MRC-000001', datetime.datetime(2024, 1, 1, 10, 0), 'POS', 'CARD_TRANSACTION', None, 'SUCCESS', 'POS', 'LAGOS', 'VERIFIED')
('2de1bee9-a0f4-488c-ba13-6eaa82b4416d', 'MRC-001396', datetime.datetime(2024, 1, 31, 0, 0, 24), 'CARD_PAYMENT', 'INVOICE_PAYMENT', Decimal('771420.08'), 'SUCCESS', 'WEB', 'ABUJA', 'VERIFIED')
('16dec229-78af-4371-a601-2bd74d362898', 'MRC-001221', datetime.datetime(2024, 1, 31, 0, 0, 37), 'POS', 'CARD_TRANSACTION', Decimal('36304.86'), 'SUCCESS', 'POS', 'IBADAN', 'PREMIUM')
('d5e58397-cb50-457a-88aa-b2aa2b0aaf26', 'MRC-005718', datetime.datetime(2024, 1, 31, 0, 0, 37), 'SAVINGS', 'DEPOSIT', Decimal('53172.09'), 'SUCCES

In [25]:
top_merchant_query = """
CREATE VIEW top_merchant AS
select merchant_id, sum(amount) AS total_volume from merchant_activity_records
where status  = 'SUCCESS'
group by merchant_id
order by total_volume desc
limit 1;
"""

# Execute the SQL
cursor.execute(top_merchant_query)
conn.commit()  # commit changes

print("View created successfully!")

View created successfully!


In [26]:
product_adoption = """
SELECT product, COUNT(DISTINCT merchant_id) AS merchant_product_count
FROM merchant_activity_records
group by product
order by merchant_product_count desc;
"""

# Execute the SQL
cursor.execute(product_adoption)
conn.commit()  # commit changes

In [ ]:
cursor.close()
conn.close()


# Create rest api that calls each view

In [27]:
from flask import Flask, jsonify, request

# Create the Flask app
app = Flask(__name__)


In [ ]:
# Define a GET route
@app.route('/analytics/top-merchant', methods=['GET'])
def greet():
    # Get optional query parameter ?name=
    name = request.args.get('name', 'World')
    return jsonify({"message": f"Hello, {name}!"})

In [32]:
# GET endpoint to fetch all users
@app.route("/analytics/top-merchant", methods=["GET"])
def get_top_merchant():
    try:
        conn = get_db_connection()
        cursor = conn.cursor()
        cursor.execute("SELECT * FROM top_merchant;")
        response = cursor.fetchall()
        cursor.close()
        conn.close()
        return jsonify(response)
    except Exception as e:
        return jsonify({"error": str(e)}), 500

In [33]:
app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with watchdog (windowsapi)


SystemExit: 1

C:\Users\sashe\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


# Automate view generation

In [37]:
import os

input_folder = "views/"

# get all files in input folder path
files = [
    input_folder+f for f in os.listdir(input_folder)
    if os.path.isfile(os.path.join(input_folder, f))
]

# clean non-numeric data in "amount" column, null "merchant_id", non-date type in "event_timestamp" and write cleaned data
for filename in files:
    with open(filename, "r") as file:
        content = file.read()
    

drop view if exists failure_rates;
CREATE VIEW failure_rates AS
SELECT product, 
    ROUND(
        100.0 * SUM(CASE WHEN status = 'FAILED' THEN 1 ELSE 0 END) / COUNT(*),
        1
    ) AS failure_rate
FROM merchant_activity_records
where status = 'SUCCESS' or status = 'FAILED'
group by product
order by failure_rate desc;
